

---

## Task 1.2: Data Cleaning and Preprocessing

**IMPORTANT NOTE ON DATA LEAKAGE:**
This notebook creates a comprehensive feature set including review-based features. These review features are ESSENTIAL for creating our target variable (`value_category`) which measures "value for money" based on rating/price ratio.

However, review-based features will be REMOVED from model input in Task 1.5 because:
1. New listings have no reviews yet
2. We need to predict value for listings without review history
3. Using reviews as input features creates data leakage

**Strategy:**
- Keep ALL features (including reviews) in this task for target creation
- Task 1.5 will filter out review-based features from X (input) while keeping them for y (target)
- Price MUST remain as a feature - you cannot predict "value for money" without knowing the price!

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import os

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("="*80)
print("T1.2: DATA CLEANING AND PREPROCESSING")
print("="*80)

In [ ]:
# Load both datasets
sf_df = pd.read_csv('../../data/raw/san francisco.csv')
sd_df = pd.read_csv('../../data/raw/san diego.csv')

# Add city identifier
sf_df['city'] = 'San Francisco'
sd_df['city'] = 'San Diego'

# Combine datasets
df = pd.concat([sf_df, sd_df], ignore_index=True)

print(f"\n Combined Dataset Shape: {df.shape}")
print(f"   - Total Rows: {df.shape[0]:,}")
print(f"   - Total Columns: {df.shape[1]}")

## 2. Missing Values Analysis

In [ ]:
print("\n" + "="*80)
print("2. MISSING VALUES ANALYSIS")
print("="*80)

# Calculate missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percentage': missing_pct.values
})

# Filter columns with missing values
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print(f"\n Columns with Missing Values: {len(missing_df)}")
print("\nTop 20 Columns with Most Missing Data:")
print(missing_df.head(20).to_string(index=False))

# Visualize missing data
plt.figure(figsize=(12, 8))
top_missing = missing_df.head(20)
plt.barh(top_missing['Column'], top_missing['Missing_Percentage'])
plt.xlabel('Missing Percentage (%)')
plt.title('Top 20 Columns with Missing Values')
plt.tight_layout()
plt.savefig('../../outputs/figures/missing_values_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Visualization saved: outputs/figures/missing_values_analysis.png")

## 3. Feature Selection - Remove Irrelevant Columns

In [ ]:
print("\n" + "="*80)
print("3. FEATURE SELECTION")
print("="*80)

# Define columns to drop (URLs, IDs, text descriptions, etc.)
columns_to_drop = [
    # URLs and IDs
    'listing_url', 'scrape_id', 'picture_url', 'host_url', 
    'host_thumbnail_url', 'host_picture_url',
    
    # Text descriptions (too noisy for initial model)
    'description', 'neighborhood_overview', 'host_about', 'name',
    
    # Redundant or highly specific
    'source', 'calendar_updated', 'last_scraped', 'calendar_last_scraped',
    
    # License (mostly missing or not useful)
    'license',
    
    # Neighbourhood group (if empty)
    'neighbourhood_group_cleansed',
    
    # Bathrooms (we'll use bathrooms_text instead)
    'bathrooms',
    
    # Host verifications (complex nested data)
    'host_verifications',
    
    # Amenities (complex nested data - can be processed later)
    'amenities'
]

# Drop columns that exist in the dataframe
columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df_cleaned = df.drop(columns=columns_to_drop)

print(f"\n Dropped {len(columns_to_drop)} columns")
print(f"   - Original: {df.shape[1]} columns")
print(f"   - After dropping: {df_cleaned.shape[1]} columns")

## 4. Data Type Conversions and Cleaning

In [ ]:
print("\n" + "="*80)
print("4. DATA TYPE CONVERSIONS")
print("="*80)

# 4.1 Clean price column (remove $ and commas)
if 'price' in df_cleaned.columns:
    df_cleaned['price'] = df_cleaned['price'].replace('[\$,]', '', regex=True).astype(float)
    print("\n Cleaned 'price' column (removed $ and commas)")

# 4.2 Convert percentage columns
percentage_cols = ['host_response_rate', 'host_acceptance_rate']
for col in percentage_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].str.rstrip('%').astype(float) / 100
        print(f" Converted '{col}' to decimal")

# 4.3 Convert boolean columns
boolean_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 
                'has_availability', 'instant_bookable']
for col in boolean_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].map({'t': 1, 'f': 0})
        print(f" Converted '{col}' to binary (0/1)")

# 4.4 Convert date columns
date_cols = ['host_since', 'first_review', 'last_review']
for col in date_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')
        print(f" Converted '{col}' to datetime")

# 4.5 Extract number from bathrooms_text
if 'bathrooms_text' in df_cleaned.columns:
    df_cleaned['bathrooms_numeric'] = df_cleaned['bathrooms_text'].str.extract('(\d+\.?\d*)').astype(float)
    print("\n✓ Extracted numeric bathrooms from 'bathrooms_text'")

## 5. Feature Engineering

In [ ]:
print("\n" + "="*80)
print("5. FEATURE ENGINEERING")
print("="*80)

# 5.1 Host experience (years as host)
if 'host_since' in df_cleaned.columns:
    df_cleaned['host_years'] = (pd.Timestamp.now() - df_cleaned['host_since']).dt.days / 365.25
    print("\n Created 'host_years' feature")

# 5.2 Days since first review
if 'first_review' in df_cleaned.columns:
    df_cleaned['days_since_first_review'] = (pd.Timestamp.now() - df_cleaned['first_review']).dt.days
    print(" Created 'days_since_first_review' feature")

# 5.3 Days since last review
if 'last_review' in df_cleaned.columns:
    df_cleaned['days_since_last_review'] = (pd.Timestamp.now() - df_cleaned['last_review']).dt.days
    print(" Created 'days_since_last_review' feature")

# 5.4 Price per person (LANDLORD-CONTROLLED)
if 'price' in df_cleaned.columns and 'accommodates' in df_cleaned.columns:
    df_cleaned['price_per_person'] = df_cleaned['price'] / df_cleaned['accommodates']
    print(" Created 'price_per_person' feature (LANDLORD-CONTROLLED)")

# 5.5 Reviews per month (REVIEW-BASED - will be removed in T1.5)
if 'reviews_per_month' not in df_cleaned.columns:
    if 'number_of_reviews' in df_cleaned.columns and 'days_since_first_review' in df_cleaned.columns:
        df_cleaned['reviews_per_month'] = (df_cleaned['number_of_reviews'] / 
                    (df_cleaned['days_since_first_review'] / 30.44))
        print(" Created 'reviews_per_month' feature (REVIEW-BASED)")

# 5.6 Availability rate (LANDLORD-CONTROLLED)
if 'availability_365' in df_cleaned.columns:
    df_cleaned['availability_rate'] = df_cleaned['availability_365'] / 365
    print(" Created 'availability_rate' feature (LANDLORD-CONTROLLED)")

# 5.7 Average review score (REVIEW-BASED - will be removed in T1.5)
review_score_cols = [col for col in df_cleaned.columns if 'review_scores_' in col and col != 'review_scores_rating']
if review_score_cols:
    df_cleaned['avg_review_score'] = df_cleaned[review_score_cols].mean(axis=1)
    print(" Created 'avg_review_score' feature (REVIEW-BASED)")

# 5.8 Has reviews flag (REVIEW-BASED - will be removed in T1.5)
if 'number_of_reviews' in df_cleaned.columns:
    df_cleaned['has_reviews'] = (df_cleaned['number_of_reviews'] > 0).astype(int)
    print(" Created 'has_reviews' feature (REVIEW-BASED)")

print(f"\n Total features after engineering: {df_cleaned.shape[1]}")

## 6. Handle Missing Values

In [ ]:
print("\n" + "="*80)
print("6. HANDLING MISSING VALUES")
print("="*80)

# 6.1 Drop columns with >50% missing values
missing_threshold = 0.5
missing_pct = df_cleaned.isnull().sum() / len(df_cleaned)
cols_to_drop = missing_pct[missing_pct > missing_threshold].index.tolist()

if cols_to_drop:
    df_cleaned = df_cleaned.drop(columns=cols_to_drop)
    print(f"\n Dropped {len(cols_to_drop)} columns with >{missing_threshold*100}% missing values")
    print(f"   Columns dropped: {cols_to_drop}")

# 6.2 Fill missing values for specific columns
# Numeric columns - fill with median
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_cleaned[col].isnull().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

print(f"\nFilled missing numeric values with median")

# Categorical columns - fill with mode or 'Unknown'
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_cleaned[col].isnull().sum() > 0:
        mode_val = df_cleaned[col].mode()
        if len(mode_val) > 0:
            df_cleaned[col].fillna(mode_val[0], inplace=True)
        else:
            df_cleaned[col].fillna('Unknown', inplace=True)

print(f"Filled missing categorical values with mode or 'Unknown'")

# Check remaining missing values
remaining_missing = df_cleaned.isnull().sum().sum()
print(f"\n Remaining missing values: {remaining_missing}")

## 7. Handle Outliers

In [ ]:
print("\n" + "="*80)
print("7. OUTLIER DETECTION AND HANDLING")
print("="*80)

# Focus on price outliers
if 'price' in df_cleaned.columns:
    # Remove listings with price = 0 or extremely high prices
    initial_rows = len(df_cleaned)
    
    # Remove price = 0
    df_cleaned = df_cleaned[df_cleaned['price'] > 0]
    
    # Remove extreme outliers (using IQR method)
    Q1 = df_cleaned['price'].quantile(0.25)
    Q3 = df_cleaned['price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    
    df_cleaned = df_cleaned[(df_cleaned['price'] >= lower_bound) & 
                    (df_cleaned['price'] <= upper_bound)]
    
    rows_removed = initial_rows - len(df_cleaned)
    print(f"\n Removed {rows_removed} rows with price outliers")
    print(f"   - Price range: ${df_cleaned['price'].min():.2f} - ${df_cleaned['price'].max():.2f}")
    print(f"   - Remaining rows: {len(df_cleaned):,}")

# Visualize price distribution after cleaning
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_cleaned['price'], bins=50, edgecolor='black')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')
plt.title('Price Distribution (After Outlier Removal)')

plt.subplot(1, 2, 2)
plt.boxplot(df_cleaned['price'])
plt.ylabel('Price ($)')
plt.title('Price Boxplot (After Outlier Removal)')

plt.tight_layout()
plt.savefig('../../outputs/figures/price_distribution_cleaned.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Visualization saved: outputs/figures/price_distribution_cleaned.png")

## 8. Create Target Variable (Value Category)

**CRITICAL:** We use review_scores_rating here to create labels. This is CORRECT because:
1. We need historical data (with reviews) to learn what makes good/bad value
2. The target represents "what value category WOULD this listing be if it had reviews"
3. Review features will be REMOVED from features(x) in Task 1.5, but kept for creating the target (y)

In [ ]:
print("\n" + "="*80)
print("8. CREATE TARGET VARIABLE - VALUE CATEGORY")
print("="*80)

# Calculate FP Score (Fair Price Score) = Rating / Price
# This measures "value for money"

if 'review_scores_rating' in df_cleaned.columns and 'price' in df_cleaned.columns:
    # Filter listings with reviews (needed for labeling)
    df_with_reviews = df_cleaned[df_cleaned['review_scores_rating'].notna()].copy()
    
    # Normalize rating (0-5 scale) and price
    df_with_reviews['rating_normalized'] = df_with_reviews['review_scores_rating'] / 20  # Convert 0-100 to 0-5
    df_with_reviews['price_normalized'] = (df_with_reviews['price'] - df_with_reviews['price'].min()) / \
                    (df_with_reviews['price'].max() - df_with_reviews['price'].min())
    
    # Calculate FP Score (higher = better value)
    df_with_reviews['fp_score'] = df_with_reviews['rating_normalized'] / (df_with_reviews['price_normalized'] + 0.1)
    
    # Classify into 3 categories based on FP Score
    fp_33 = df_with_reviews['fp_score'].quantile(0.33)
    fp_67 = df_with_reviews['fp_score'].quantile(0.67)
    
    def classify_value(fp_score):
        if fp_score <= fp_33:
            return 'Poor_Value'
        elif fp_score <= fp_67:
            return 'Fair_Value'
        else:
            return 'Excellent_Value'
    
    df_with_reviews['value_category'] = df_with_reviews['fp_score'].apply(classify_value)
    
    print(f"\n Created FP Score and Value Category")
    print(f"   - Listings with reviews: {len(df_with_reviews):,}")
    print(f"   - FP Score range: {df_with_reviews['fp_score'].min():.2f} - {df_with_reviews['fp_score'].max():.2f}")
    print(f"\n Value Category Distribution:")
    print(df_with_reviews['value_category'].value_counts())
    
    # Visualize distribution
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    df_with_reviews['value_category'].value_counts().plot(kind='bar', color=['red', 'orange', 'green'])
    plt.xlabel('Value Category')
    plt.ylabel('Count')
    plt.title('Distribution of Value Categories')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    plt.hist(df_with_reviews['fp_score'], bins=50, edgecolor='black')
    plt.xlabel('FP Score')
    plt.ylabel('Frequency')
    plt.title('FP Score Distribution')
    
    plt.tight_layout()
    plt.savefig('../../outputs/figures/value_category_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n Visualization saved: outputs/figures/value_category_distribution.png")
    
    # Use df_with_reviews for further processing
    df_final = df_with_reviews.copy()
else:
    print("\n Warning: 'review_scores_rating' or 'price' not found. Skipping target creation.")
    df_final = df_cleaned.copy()

## 9. Save Cleaned Data (Before Train-Test Split)

**Note:** This dataset contains ALL features including review-based ones.
Task 1.3 and 1.4 will use this file.
Task 1.5 will filter out review-based features from model input.

In [ ]:
print("\n" + "="*80)
print("9. SAVE CLEANED DATA")
print("="*80)

# Create output directory if it doesn't exist
os.makedirs('../../data/processed', exist_ok=True)

# Save cleaned full dataset with target
df_final.to_csv('../../data/processed/listings_cleaned_with_target.csv', index=False)
print("\n Saved: data/processed/listings_cleaned_with_target.csv")
print(f"   - Shape: {df_final.shape}")
print(f"   - Contains ALL features (including review-based)")
print(f"   - Review features will be filtered in Task 1.5")

## 10. Summary Report

In [ ]:
print("\n" + "="*80)
print("10. PREPROCESSING SUMMARY REPORT")
print("="*80)

summary = f"""
DATA PREPROCESSING COMPLETED SUCCESSFULLY!
{'='*80}

ORIGINAL DATA:
  - San Francisco: 7,780 listings
  - San Diego: 13,162 listings
  - Combined: {df.shape[0]:,} listings, {df.shape[1]} columns

AFTER CLEANING:
  - Final dataset: {df_final.shape[0]:,} listings, {df_final.shape[1]} columns

TARGET VARIABLE:
  - Name: value_category
  - Classes: Poor_Value, Fair_Value, Excellent_Value
  - Based on FP Score = Rating / Price (measures value for money)
  - Distribution:
{df_final['value_category'].value_counts().to_string()}

KEY STEPS PERFORMED:
   Removed irrelevant columns (URLs, IDs, text descriptions)
   Converted data types (price, percentages, booleans, dates)
   Feature engineering (host_years, price_per_person, etc.)
   Handled missing values (dropped >50% missing, imputed rest)
   Removed outliers (price outliers using IQR method)
   Created target variable (FP Score classification)

 IMPORTANT - DATA LEAKAGE PREVENTION:
  - This dataset contains review-based features (number_of_reviews, review_scores, etc.)
  - These features were KEPT for creating the target variable (value_category)
  - Task 1.5 will REMOVE review-based features from model input (X)
  - Price MUST remain as a feature - cannot predict value without knowing price!

OUTPUT FILES:
  - data/processed/listings_cleaned_with_target.csv

VISUALIZATIONS:
  - outputs/figures/missing_values_analysis.png
  - outputs/figures/price_distribution_cleaned.png
  - outputs/figures/value_category_distribution.png


{'='*80}
Task 1.2 COMPLETED!
{'='*80}
"""

print(summary)

# Save summary to file
os.makedirs('../../outputs/reports', exist_ok=True)
with open('../../outputs/reports/T1.2_preprocessing_summary.txt', 'w') as f:
    f.write(summary)

print("\n Summary saved to: outputs/reports/T1.2_preprocessing_summary.txt")